In [1]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [2]:
class DNN(Model):
    def __init__(self,hidden_dim,output_dim):
        super().__init__()
        self.l1=Dense(hidden_dim,activation='tanh')
        self.l2=Dense(hidden_dim,activation='tanh')
        self.l3=Dense(hidden_dim,activation='tanh')
        self.l4=Dense(output_dim,activation='softmax')
        self.ls=[self.l1,self.l2,self.l3,self.l4]
        
    def call(self,x):
        for layer in self.ls:
            x=layer(x)
        return x

In [3]:
mnist=datasets.mnist
(x_train,t_train),(x_test,t_test)=mnist.load_data()

x_train=(x_train.reshape(-1,784)/255).astype(np.float32)
x_test=(x_test.reshape(-1,784)/255).astype(np.float32)
t_train=np.eye(10)[t_train].astype(np.float32)
t_test=np.eye(10)[t_test].astype(np.float32)

In [4]:
model=DNN(200,10)

In [5]:
criterion=losses.CategoricalCrossentropy()
optimizer=optimizers.SGD(learning_rate=0.1)
train_loss=metrics.Mean()
train_acc=metrics.CategoricalAccuracy()

def compute_loss(t,y):
    return criterion(t,y)

def train_step(x,t):
    with tf.GradientTape() as tape:
        preds=model(x)
        loss=compute_loss(preds,t)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss)
    train_acc(t,preds)
    return loss

epochs=30
batch_size=100
n_batches=x_train.shape[0]//batch_size

for epoch in range(epochs):
    x_,t_=shuffle(x_train,t_train)
    
    for batch in range(n_batches):
        start=batch*batch_size
        end=start+batch_size
        train_step(x_[start:end],t_[start:end])
    
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}'.format(epoch,train_loss.result(),train_acc.result()))


epoch: 0, loss: 2.022, acc: 0.882
epoch: 1, loss: 1.507, acc: 0.911
epoch: 2, loss: 1.264, acc: 0.925
epoch: 3, loss: 1.111, acc: 0.934
epoch: 4, loss: 1.005, acc: 0.940
epoch: 5, loss: 0.923, acc: 0.945
epoch: 6, loss: 0.855, acc: 0.949
epoch: 7, loss: 0.799, acc: 0.953
epoch: 8, loss: 0.752, acc: 0.955
epoch: 9, loss: 0.710, acc: 0.958
epoch: 10, loss: 0.675, acc: 0.960
epoch: 11, loss: 0.643, acc: 0.962
epoch: 12, loss: 0.613, acc: 0.964
epoch: 13, loss: 0.587, acc: 0.965
epoch: 14, loss: 0.564, acc: 0.966
epoch: 15, loss: 0.542, acc: 0.968
epoch: 16, loss: 0.521, acc: 0.969
epoch: 17, loss: 0.501, acc: 0.970
epoch: 18, loss: 0.483, acc: 0.971
epoch: 19, loss: 0.467, acc: 0.972
epoch: 20, loss: 0.452, acc: 0.973
epoch: 21, loss: 0.438, acc: 0.974
epoch: 22, loss: 0.424, acc: 0.975
epoch: 23, loss: 0.412, acc: 0.976
epoch: 24, loss: 0.400, acc: 0.976
epoch: 25, loss: 0.388, acc: 0.977
epoch: 26, loss: 0.378, acc: 0.978
epoch: 27, loss: 0.368, acc: 0.978
epoch: 28, loss: 0.359, acc: 0

In [6]:
test_loss=metrics.Mean()
test_acc=metrics.CategoricalAccuracy()

def test_step(x,t):
    preds=model(x)
    loss=compute_loss(t,preds)
    test_loss(loss)
    test_acc(t,preds)
    return loss

test_step(x_test,t_test)

print('test_loss: {:.3f}, test_acc:{:.3f}'.format(test_loss.result(),test_acc.result()))

test_loss: 0.168, test_acc:0.977
